In [43]:
import json
from pymystem3 import Mystem

text = "Красивая Энди красиво мыла раму"
m = Mystem()
lemmas = m.lemmatize(text)

print("lemmas:", ''.join(lemmas))
arr = m.analyze(''.join(lemmas))
arr

lemmas: красивый энди красиво мыть рама



[{'analysis': [{'lex': 'красивый',
    'wt': 1,
    'gr': 'A=(вин,ед,полн,муж,неод|им,ед,полн,муж)'}],
  'text': 'красивый'},
 {'text': ' '},
 {'analysis': [{'lex': 'энди',
    'wt': 1,
    'gr': 'S,имя,муж,од=(пр,мн|пр,ед|вин,мн|вин,ед|дат,мн|дат,ед|род,мн|род,ед|твор,мн|твор,ед|им,мн|им,ед)'}],
  'text': 'энди'},
 {'text': ' '},
 {'analysis': [{'lex': 'красиво', 'wt': 0.8149252476, 'gr': 'ADV='}],
  'text': 'красиво'},
 {'text': ' '},
 {'analysis': [{'lex': 'мыть', 'wt': 1, 'gr': 'V,несов,пе=инф'}],
  'text': 'мыть'},
 {'text': ' '},
 {'analysis': [{'lex': 'рама', 'wt': 0.9942164394, 'gr': 'S,жен,неод=им,ед'}],
  'text': 'рама'},
 {'text': '\n'}]

In [16]:
import torch
import string
import gensim.downloader as api
import numpy as np
from torch import nn
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")


class SpamToxicDetector:
    def __init__(self, model, tokenizer, word2vec, idx2label):
        self.model = model
        self.tokenizer = tokenizer
        self.idx2label = idx2label
        self.word2vec = word2vec
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def get_tokens(self, text):
        return [token for token in self.tokenizer.tokenize(text.lower()) if token not in russian_stopwords and
                all(symbol not in string.punctuation for symbol in token) and len(token) >= 3]

    def get_avg_embedding(self, tokens):
        embedding = [(self.word2vec[token] - self.mean) / self.std for token in tokens if token in self.word2vec]

        if len(embedding) == 0:
            embedding = np.zeros(self.word2vec.vector_size)
        else:
            embedding = np.mean(embedding, axis=0)
        return embedding

    def make_prediction(self, text):
        tokens = self.get_tokens(text)
        embedding = self.get_avg_embedding(tokens)
        pred = self.model(torch.tensor(embedding).float())
        pred_label_idx = torch.argmax(pred).item()
        return self.idx2label[pred_label_idx]


def load_w2v():
    word2vec = api.load("word2vec-ruscorpora-300")
    return word2vec


def load_tokenizer():
    return WordPunctTokenizer()


def load_model(path, embed_size, num_classes):
    model = nn.Sequential(
                      nn.Linear(300, 64),
                      nn.ReLU(),
                      nn.Dropout(0.1),
                      nn.Linear(64, 3),
                      nn.LogSoftmax(dim=1)
    )
    model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))
    return model



In [17]:
idx2label = {0: 'bad', 1: 'good', 2: 'neutral'}
num_classes = len(idx2label)
word2vec = load_w2v()
tokenizer = load_tokenizer()
model = load_model('./model.pt', word2vec.vector_size, num_classes)

classifier = SpamToxicDetector(model, tokenizer, word2vec, idx2label)
text = 'я люблю этот фильм но в конце концовку слили в целом'
classifier.make_prediction(text)

RuntimeError: Error(s) in loading state_dict for Sequential:
	size mismatch for 0.weight: copying a param with shape torch.Size([64, 34473]) from checkpoint, the shape in current model is torch.Size([64, 300]).